# S3DIS examples
- This code is examples of `torchpcp.datasets.S3DIS.S3DIS`.

## install a package
- [k3d](https://github.com/K3D-tools/K3D-jupyter): visualizer

## import packages and define values
- dataset_path: dataset path of [S3DIS](https://openaccess.thecvf.com/content_cvpr_2016/papers/Armeni_3D_Semantic_Parsing_CVPR_2016_paper.pdf)

In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys
# BASE_DIR = os.path.dirname(os.path.abspath(__file__)) # for .py
BASE_DIR = os.path.abspath('') # for .ipynb
sys.path.append(os.path.abspath(os.path.join(BASE_DIR, "../../"))) # for torchpcp package path

from torchpcp.datasets.S3DIS import S3DIS
from torchpcp.datasets.S3DIS.utils.indoor3d_util import room2blocks_wrapper_normalized
from torchpcp.datasets.ASIS.S3DIS import Preprocessing

from torchpcp.utils.io.ply import write_pc_label
import k3d
import numpy as np


## preprocess S3DIS dataset
- In order to use the dataset, you need to preprocess data of `Stanford3dDataset_v1.2_Aligned_Version`.
- This preprocessing is the same method that [WXinlong/ASIS](https://github.com/WXinlong/ASIS) utilizes.
- I call preprocessed datasets S3DIS scene and block dataset. 
- The difference between these preprocessed datasets is below.
    - S3DIS scene dataset : Dataset including coords, colors, semantic and instance label in scene (e.g. room).
    - S3DIS block dataset : Dataset including divided data (block) for the input of deep learning model (ASIS, JSNet ... etc.).

In [2]:
# define dataset path
dataset_path = "/mnt/databox/datasets/S3DIS/Stanford3dDataset_v1.2_Aligned_Version/" # S3DIS path
output_path = "data/" # output (preprocessed dataset) path

# scene dataset path
scene_output_path = os.path.join(output_path, "scenes")

# block dataset path
block_output_path = os.path.join(output_path, "blocks")

In [6]:
# create S3DIS scene dataset
# make a dir.
os.makedirs(scene_output_path, exist_ok=True)
# create a scene dataset
Preprocessing.create_scene_dataset(dataset_path, scene_output_path)
print("Finish creating S3DIS scene dataset.")

NameError: name 'Preprocessing' is not defined

In [7]:
# S3DIS block dataset
# define paths and params
num_points = 4096
block_size = 1.0
stride = 0.5
# make a dir.
os.makedirs(block_output_path, exist_ok=True)
# create a block dataset
Preprocessing.create_block_dataset(scene_output_path, block_output_path, 
                                   num_points, block_size, stride)
print("Finish creating S3DIS block dataset.")

NameError: name 'Preprocessing' is not defined

## use S3DIS block dataset
- `dataset_path` is path of S3DIS dataset dir created by `torchpcp.datasets.S3DIS.S3DIS.Preprocessing.create_block_dataset`.

In [3]:
train_dataset = S3DIS.S3DISBlockDataset(
    dataset_path="/mnt/databox/datasets/S3DIS/ASIS/S3DIS/4096/blocks/",
    num_points=4096,
    split="train",
    test_area=5
)

# get block data
point_cloud, sem_labels, ins_labels, datainfo = train_dataset[0]

print("point cloud shape:", point_cloud.shape)
print("semantic label shape:", sem_labels.shape)
print("instance label shape:", ins_labels.shape)
print("scene data path and block index:", datainfo)

# get coords
coords = point_cloud[:, 0:3]
# get color codes
colors = point_cloud[:, 3:6] * 255
colors = colors.astype(np.int32)
colors = colors[:, 0]*256*256 + colors[:,1]*256 + colors[:,2] # to color code

# visualize point cloud
plot = k3d.plot()
points = k3d.points(coords, colors.astype(np.float32), point_size=0.02, shader='flat')
plot += points
plot.display()


point cloud shape: (4096, 9)
semantic label shape: (4096,)
instance label shape: (4096,)
scene data path and block index: ['/mnt/databox/datasets/S3DIS/ASIS/S3DIS/4096/blocks/Area_1_WC_1.h5', 0]


/home/coder/anaconda3/envs/py37/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## define S3DIS scene dataset
- `dataset_path` is a path of S3DIS dataset dir created by `torchpcp.datasets.S3DIS.S3DIS.Preprocessing.create_scene_dataset`.

In [35]:
train_dataset = S3DIS.S3DISSceneDataset(
    dataset_path="/mnt/databox/datasets/S3DIS/ASIS/S3DIS/4096/scenes/",
    split="train",
    test_area=5
)

# get block data
point_cloud, sem_labels, ins_labels, scene_path = train_dataset[2]

print("point cloud shape:", point_cloud.shape)
print("semantic label shape:", sem_labels.shape)
print("instance label shape:", ins_labels.shape)
print("scene data path:", scene_path)

# get coords
coords = point_cloud[:, 0:3]
# get color codes
colors = point_cloud[:, 3:6]
colors = colors.astype(np.int32)
colors = colors[:, 0]*256*256 + colors[:,1]*256 + colors[:,2] # to color code

# visualize point cloud
plot = k3d.plot()
points = k3d.points(coords, colors.astype(np.float32), point_size=0.02, shader='flat')
plot += points
plot.display()


point cloud shape: (1535040, 6)
semantic label shape: (1535040,)
instance label shape: (1535040,)
scene data path: /mnt/databox/datasets/S3DIS/ASIS/S3DIS/4096/scenes/Area_1_conferenceRoom_2.npy


Output()

In [31]:
# get boundary
from torchpcp.modules import functional as F
import torch

batch_coords = torch.tensor([coords], dtype=torch.float32, device="cuda")
batch_coords = torch.transpose(batch_coords, 1, 2)
k = 32

bq_coords_indexes = F.nns.ball_query(batch_coords, batch_coords, 0.1, k)


In [32]:
source = bq_coords_indexes[:,:,0:1].repeat(1,1,k-1) # get center points of kNN
target = bq_coords_indexes[:,:,1:] # remove center points of kNN

row_source = F.other.index2row(source)
row_target = F.other.index2row(target)

row_ins_labels = torch.tensor([ins_labels]).view(-1)

row_source_labels = row_ins_labels[row_source]
row_target_labels = row_ins_labels[row_target]

transition = row_source_labels != row_target_labels


In [33]:
B, C, N = batch_coords.shape
bt = transition.view(B, N, -1).to(dtype=torch.long)
bt, _ = torch.max(bt, dim=-1)


In [34]:
from torchpcp.utils import converter
from torchpcp.utils.pytorch_tools import t2n

# print(bt.shape)

colors_bt = converter.label_to_color(t2n(bt[0]))
colors_bt = colors_bt[:, 0]*256*256 + colors_bt[:,1]*256 + colors_bt[:,2] # to color code
# print(coords.shape)
# print(colors_bt.shape)
# visualize point cloud
plot = k3d.plot()
points = k3d.points(coords, colors_bt.astype(np.float32), point_size=0.02, shader='flat')
plot += points
plot.display()


Output()

In [11]:
# get block point clouds from scene data
num_points = 4096
block_size = 1.0
stride = 0.5
point_clouds, sem_labels, ins_labels = room2blocks_wrapper_normalized(scene_path, num_points, block_size=block_size, stride=stride, random_sample=False, sample_num=None)

print("point cloud shape:", point_clouds.shape)
print("semantic label shape:", sem_labels.shape)
print("instance label shape:", ins_labels.shape)

# get a block point cloud
point_cloud = point_clouds[0]
# get coords
coords = point_cloud[:, 0:3]
# get color codes
colors = point_cloud[:, 3:6] * 255
colors = colors.astype(np.int32)
colors = colors[:, 0]*256*256 + colors[:,1]*256 + colors[:,2] # to color code

# visualize point cloud
plot = k3d.plot()
points = k3d.points(coords, colors.astype(np.float32), point_size=0.02, shader='flat')
plot += points
plot.display()



point cloud shape: (132, 4096, 9)
semantic label shape: (132, 4096)
instance label shape: (132, 4096)


Output()

## assign labels to the point cloud

In [10]:
train_dataset = S3DIS.S3DISSceneDataset(
    dataset_path="/mnt/databox/datasets/S3DIS/ASIS/S3DIS/4096/scenes/",
    split="train",
    test_area=5
)

# get block data
point_cloud, sem_labels, ins_labels, scene_path = train_dataset[2]

# get coords
coords = point_cloud[:, 0:3]
# create label colors
colors = label_to_color(ins_labels.astype(np.int32))
# colors = point_cloud[:, 3:6]
colors = colors.astype(np.int32)
colors = colors[:, 0]*256*256 + colors[:,1]*256 + colors[:,2] # to color code

# visualize point cloud
plot = k3d.plot()
points = k3d.points(coords, colors.astype(np.float32), point_size=0.02, shader='flat')
plot += points
plot.display()


Output()